In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Predict the state of a loan given some information about it

In [2]:
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

y2015.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## Blind Approach

Try putting everything into a Random Forest without much preprocessing. SKLearn needs independent variables to be numeric, use get_dummies to generate a dummy variable for every categorical column

In [3]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status',1)
Y = y2015['loan_status']
#X = pd.get_dummies(X) #this kills the kernel

#cross_val_score(rfc,X,Y,cv=5)

## Data Cleaning

- The get_dummies function can be very memory intensive, particularly if data is poorly typed (see earlier warning)
- Mixed data types get converted to objects which could create problems
- In this 400,000 row dataset, a bad type means its going to see 400k distinct values and try and create dummies for all of them

In [4]:
#look at categorical variables and see how many distinct counts there are
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


__Result:__ Some of these variables have over 100,000 distinct types which is causing the problem

- Drop ones with over 30 unique values
- Convert to numeric where it makes sense
- Could extract numeric features from dates, but here just drop them since there is a lot of data

In [5]:
#convert id and interest rate to numeric
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

#drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [6]:
#find what is causing dtype error on id column (should be all integers)
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#remove last 2 rows that don't contain any data
y2015 = y2015[:-2]

#try get_dummies function again
pd.get_dummies(y2015).head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0


## Second Attempt

- Had to sacrifice dub grade, state address, and description variables
- If desired, can run dummies independently and append them back into dataframe
- Drop NA columns rather than impute, have enough data to most likely get away with this

In [8]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status',1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc,X,Y,cv=10)

array([0.97960153, 0.9801952 , 0.98100259, 0.98195246, 0.9706483 ,
       0.97872239, 0.92101451, 0.98076421, 0.97995583, 0.9801691 ])

__Result:__ cross validation reports about 97-98% accuracy

__Potential problems:__
- Didn't do much feature selection or model refinement, keeping many unnecessary features
- Scores do have variance, can try to correct by increasing number of estimators, but will run even slower

## Drill: Third Attempt

Get rid of as much data as possible without dropping below μ = 90% accuracy. This involves looking into the data to determine which features are most important. Can be raw features or generated dummies, using PCA/correlation matrices are helpful here.

Try doing this without using anything related to payment amount or outstanding principal

In [68]:
#drop variables related to payments/principle
X_2 = X.drop(['out_prncp',
              'out_prncp_inv',
              'total_pymnt',
              'total_pymnt_inv',
              'last_pymnt_amnt',
              'funded_amnt',
              'funded_amnt_inv',
              'installment',
              'loan_amnt',
              'tot_coll_amt',
              'tot_cur_bal'],1)
X_2 = pd.get_dummies(X_2)
X_2 = X_2.dropna(axis=1)
cross_val_score(rfc,X_2,Y,cv=10)

array([0.94170169, 0.96240887, 0.97428225, 0.97314241, 0.964474  ,
       0.96509143, 0.93854045, 0.96746539, 0.94929584, 0.96169192])

In [74]:
#drop variables that seem unlikely to predict loan status
#drop issue date
X_3 = X_2.drop(['issue_d_Apr-2015',
                'issue_d_Aug-2015',
                'issue_d_Dec-2015',
                'issue_d_Feb-2015',
                'issue_d_Jan-2015',
                'issue_d_Jul-2015',
                'issue_d_Jun-2015',
                'issue_d_Mar-2015',
                'issue_d_May-2015',
                'issue_d_Nov-2015',
                'issue_d_Oct-2015',
                'issue_d_Sep-2015'],1)
X_3 = pd.get_dummies(X_3)
X_3 = X_3.dropna(axis=1)
cross_val_score(rfc,X_3,Y,cv=10)

array([0.9526727 , 0.97691814, 0.97706063, 0.97608701, 0.97539777,
       0.97577773, 0.97428103, 0.97458976, 0.97252238, 0.97183299])

In [77]:
#drop payment due dates, credit pull dates
X_4 = X_3.drop(['last_pymnt_d_Apr-2015',
                'last_pymnt_d_Apr-2016',
                'last_pymnt_d_Aug-2015',
                'last_pymnt_d_Aug-2016',
                'last_pymnt_d_Dec-2015',
                'last_pymnt_d_Dec-2016',
                'last_pymnt_d_Feb-2015',
                'last_pymnt_d_Feb-2016',
                'last_pymnt_d_Jan-2015',
                'last_pymnt_d_Jan-2016',
                'last_pymnt_d_Jan-2017',
                'last_pymnt_d_Jul-2015',
                'last_pymnt_d_Jul-2016',
                'last_pymnt_d_Jun-2015',
                'last_pymnt_d_Jun-2016',
                'last_pymnt_d_Mar-2015',
                'last_pymnt_d_Mar-2016',
                'last_pymnt_d_May-2015',
                'last_pymnt_d_May-2016',
                'last_pymnt_d_Nov-2015',
                'last_pymnt_d_Nov-2016',
                'last_pymnt_d_Oct-2015',
                'last_pymnt_d_Oct-2016',
                'last_pymnt_d_Sep-2015',
                'last_pymnt_d_Sep-2016',
                'next_pymnt_d_Feb-2017',
                'next_pymnt_d_Jan-2017',
                'next_pymnt_d_Jul-2016',
                'next_pymnt_d_Mar-2017',
                'last_credit_pull_d_Apr-2015',
                'last_credit_pull_d_Apr-2016',
                'last_credit_pull_d_Aug-2015',
                'last_credit_pull_d_Aug-2016',
                'last_credit_pull_d_Dec-2014',
                'last_credit_pull_d_Dec-2015',
                'last_credit_pull_d_Dec-2016',
                'last_credit_pull_d_Feb-2015',
                'last_credit_pull_d_Feb-2016',
                'last_credit_pull_d_Jan-2015',
                'last_credit_pull_d_Jan-2016',
                'last_credit_pull_d_Jan-2017',
                'last_credit_pull_d_Jul-2015',
                'last_credit_pull_d_Jul-2016',
                'last_credit_pull_d_Jun-2015',
                'last_credit_pull_d_Jun-2016',
                'last_credit_pull_d_Mar-2015',
                'last_credit_pull_d_Mar-2016',
                'last_credit_pull_d_May-2015',
                'last_credit_pull_d_May-2016',
                'last_credit_pull_d_Nov-2015',
                'last_credit_pull_d_Nov-2016',
                'last_credit_pull_d_Oct-2015',
                'last_credit_pull_d_Oct-2016',
                'last_credit_pull_d_Sep-2015',
                'last_credit_pull_d_Sep-2016'],1)
X_4 = pd.get_dummies(X_4)
X_4 = X_4.dropna(axis=1)
cross_val_score(rfc,X_4,Y,cv=10)

array([0.7346774 , 0.85611835, 0.84697585, 0.84377004, 0.82244123,
       0.61025885, 0.65480064, 0.57142179, 0.56135559, 0.52189712])

In [78]:
#too far, try putting credit pulls back in
X_5 = X_3.drop(['last_pymnt_d_Apr-2015',
                'last_pymnt_d_Apr-2016',
                'last_pymnt_d_Aug-2015',
                'last_pymnt_d_Aug-2016',
                'last_pymnt_d_Dec-2015',
                'last_pymnt_d_Dec-2016',
                'last_pymnt_d_Feb-2015',
                'last_pymnt_d_Feb-2016',
                'last_pymnt_d_Jan-2015',
                'last_pymnt_d_Jan-2016',
                'last_pymnt_d_Jan-2017',
                'last_pymnt_d_Jul-2015',
                'last_pymnt_d_Jul-2016',
                'last_pymnt_d_Jun-2015',
                'last_pymnt_d_Jun-2016',
                'last_pymnt_d_Mar-2015',
                'last_pymnt_d_Mar-2016',
                'last_pymnt_d_May-2015',
                'last_pymnt_d_May-2016',
                'last_pymnt_d_Nov-2015',
                'last_pymnt_d_Nov-2016',
                'last_pymnt_d_Oct-2015',
                'last_pymnt_d_Oct-2016',
                'last_pymnt_d_Sep-2015',
                'last_pymnt_d_Sep-2016',
                'next_pymnt_d_Feb-2017',
                'next_pymnt_d_Jan-2017',
                'next_pymnt_d_Jul-2016',
                'next_pymnt_d_Mar-2017',],1)
X_5 = pd.get_dummies(X_5)
X_5 = X_5.dropna(axis=1)
cross_val_score(rfc,X_5,Y,cv=10)

array([0.78544798, 0.85927667, 0.87399967, 0.86713685, 0.86984089,
       0.78408929, 0.71352917, 0.66004892, 0.70617712, 0.60509191])

In [88]:
#still too far, try a different group of variables
#customer entered title descriptions
X_6 = X_3.drop(['title_Car financing',
                'title_Credit Card/Auto Repair',
                'title_Credit card refinancing',
                'title_Debt consolidation',
                'title_DebtC',
                'title_Green loan',
                'title_Home buying',
                'title_Home improvement',
                'title_Learning and training',
                'title_Major purchase',
                'title_Medical expenses',
                'title_Moving and relocation',
                'title_New Baby and New House (CC Consolidate)',
                'title_Other',
                'title_Pay off Lowes Card',
                'title_Paying off higher interest cards & auto',
                'title_Prescription Drug and Medical Costs',
                'title_SAVE',
                'title_Simple Loan Until Contract Is Completed',
                'title_Student Loan',
                'title_Trying to come back to reality!',
                'title_Vacation',
                'title_considerate',
                'title_new day',
                'title_new kitchen for momma!',
                'title_odymeds',
                'title_Business'],1)
X_6 = pd.get_dummies(X_6)
X_6 = X_6.dropna(axis=1)
cross_val_score(rfc,X_6,Y,cv=10)

array([0.95198404, 0.974306  , 0.97668068, 0.97309492, 0.97506531,
       0.97525528, 0.97259493, 0.97209623, 0.97109744, 0.97259298])

In [87]:
#drop misc variables
X_7 = X_6.drop(['policy_code',
                'verification_status_Not Verified',
                'verification_status_Source Verified',
                'verification_status_Verified',
                'delinq_2yrs',
                'inq_last_6mths',
                'open_acc',
                'pub_rec',
                'revol_bal',
                'total_acc',
                'total_rec_prncp',
                'total_rec_int',
                'total_rec_late_fee',
                'recoveries',
                'collection_recovery_fee',
                'collections_12_mths_ex_med',
                'next_pymnt_d_Feb-2017',
                'next_pymnt_d_Jan-2017',
                'next_pymnt_d_Jul-2016',
                'next_pymnt_d_Mar-2017'],1)
X_7 = pd.get_dummies(X_7)
X_7 = X_7.dropna(axis=1)
cross_val_score(rfc,X_7,Y,cv=10)

array([0.83310774, 0.88354587, 0.86915533, 0.88093372, 0.89180717,
       0.88627404, 0.87180888, 0.88643758, 0.89445935, 0.88058709])